In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop = stopwords.words()
import pickle

/raid/alebedev/myenvtr/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/raid/alebedev/myenvtr/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='la

In [2]:
train_df = pd.read_pickle("train.pkl")

In [3]:
train_df

,ID,url,title,label,url_len,pri_domain,letters_count,digits_count,special_chars_count,url_region,root_domain
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",0,7,m.kp.md,5,0,2,Moldova,kp
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,0,5,www.kp.by,7,0,2,Belarus,kp
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,0,13,fanserials.tv,12,0,1,Tuvalu,fanserials
3,3,colorbox.spb.ru,Не Беси Меня Картинки,0,15,colorbox.spb.ru,13,0,2,Russia,spb
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,0,13,tula-sport.ru,11,0,2,Russia,tula-sport
...,...,...,...,...,...,...,...,...,...,...,...
135304,135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,0,7,mail.ru,6,0,1,Russia,mail
135305,135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....,0,6,www.ntv.ru,8,0,2,Russia,ntv
135306,135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati...",0,25,topclassiccarsforsale.com,24,0,1,Global,topclassiccarsforsale
135307,135307,wowcream.ru,Купить It's Skin Сыворотка питательная Power 1...,0,11,wowcream.ru,10,0,1,Russia,wowcream


In [4]:
test_df = pd.read_pickle("test.pkl")

In [5]:
test_df

,ID,url,title,url_len,pri_domain,letters_count,digits_count,special_chars_count,url_region,root_domain
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...,13,www.kommersant.ru,15,0,2,Russia,kommersant
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ...",15,urexpert.online,14,0,1,Global,urexpert
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12,13,imperimeha.ru,12,0,1,Russia,imperimeha
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ...",17,national-porn.com,15,0,2,Global,national-porn
4,135313,2gis.ru,67,7,2gis.ru,5,1,1,Russia,2gis
...,...,...,...,...,...,...,...,...,...,...
165373,300682,etp.armtek.ru,Armtek - запчасти для грузовых и легковых авто...,13,etp.armtek.ru,11,0,2,Russia,armtek
165374,300683,mail.ru,"Лилия Якупова - Караганда, Карагандинская обла...",7,mail.ru,6,0,1,Russia,mail
165375,300684,xn----8sbnqchpeeeth.xn--p1ai,Администрация Лесного района Тверской области ...,28,xn----8sbnqchpeeeth.xn--p1ai,19,2,7,Global,xn----8sbnqchpeeeth
165376,300685,www-sunhome-ru.cdn.ampproject.org,Сонник Изменение сознания. К чему снится Измен...,33,www-sunhome-ru.cdn.ampproject.org,28,0,5,Global,ampproject


In [6]:
y_train = train_df["label"]
y_train

0         0
1         0
2         0
3         0
4         0
         ..
135304    0
135305    0
135306    0
135307    0
135308    0
Name: label, Length: 135309, dtype: int64

In [7]:
X_train = train_df
del X_train["label"]
X_train

,ID,url,title,url_len,pri_domain,letters_count,digits_count,special_chars_count,url_region,root_domain
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",7,m.kp.md,5,0,2,Moldova,kp
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,5,www.kp.by,7,0,2,Belarus,kp
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,13,fanserials.tv,12,0,1,Tuvalu,fanserials
3,3,colorbox.spb.ru,Не Беси Меня Картинки,15,colorbox.spb.ru,13,0,2,Russia,spb
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,13,tula-sport.ru,11,0,2,Russia,tula-sport
...,...,...,...,...,...,...,...,...,...,...
135304,135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,7,mail.ru,6,0,1,Russia,mail
135305,135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....,6,www.ntv.ru,8,0,2,Russia,ntv
135306,135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati...",25,topclassiccarsforsale.com,24,0,1,Global,topclassiccarsforsale
135307,135307,wowcream.ru,Купить It's Skin Сыворотка питательная Power 1...,11,wowcream.ru,10,0,1,Russia,wowcream


In [8]:
del X_train["ID"]

In [9]:
del X_train["url"]

In [10]:
X_train["pri_domain"] = X_train["pri_domain"].apply(lambda x: str(x))

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
from sklearn.preprocessing import RobustScaler

In [12]:
num_pipeline = Pipeline([
    ('scaler', RobustScaler())
])

In [21]:
num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

In [12]:
#best
num_pipeline = Pipeline([
    ('scaler', MinMaxScaler())
])

In [20]:
country_pipeline = Pipeline([
    ('encoder', OneHotEncoder()) 
])

In [37]:
#best
num_pipeline = Pipeline([
    ('scaler', MaxAbsScaler())
])

NameError: name 'MaxAbsScaler' is not defined

In [44]:
 from sklearn.ensemble import ExtraTreesRegressor

In [23]:
from catboost import CatBoostClassifier

In [36]:
from catboost import CatBoostRegressor

In [13]:
from sklearn.svm import LinearSVC

In [51]:
from sklearn.ensemble import RandomForestClassifier

In [58]:
from sklearn.naive_bayes import MultinomialNB

In [66]:
from sklearn.linear_model import SGDClassifier

In [74]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
SEED = 42
pipeline = Pipeline(steps=[
    (
        'features', 
        ColumnTransformer([
#             ('url', TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4), min_df=3, max_df=0.75), 'url'),
            ('pri_domain', TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4), min_df=3, max_df=0.75), 'pri_domain'),
            ('root_domain', TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4), min_df=3, max_df=0.75), 'root_domain'),
            ('url_region', CountVectorizer(), 'url_region'),
#             ('url_region', 'passthrough', ["url_region"])
            ('url_len', num_pipeline, ['url_len']),
            ('letters_count', num_pipeline, ['letters_count']),
            ('digits_count', num_pipeline, ['digits_count']),
            ('special_chars_count', num_pipeline, ['special_chars_count']),
            ('title', CountVectorizer(stop_words=stop), 'title')
        ])
    ),
#     ('clf', LogisticRegression(random_state=SEED))
#     ('clf', SGDClassifier(random_state=SEED))
#     ('clf', SGDClassifier(random_state=SEED))
#     ('clf', MultinomialNB())
#     ('clf', RandomForestClassifier(random_state=SEED))
    ('clf', LinearSVC(random_state=SEED)) #gridSearchCV need
#     ('clf', CatBoostClassifier(random_seed=SEED, devices="cuda"))
#     ('clf', ExtraTreesRegressor(random_state=SEED))
#     ('clf', LogisticRegression(random_state=SEED, solver='lbfgs', class_weight='balanced')) #best
])



In [ ]:
pipeline.fit(X_train, y_train)

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
param_grid = {
    'features__pri_domain__min_df': [1, 3],
    'features__pri_domain__max_df': [0.5, 0.75],
    'features__root_domain__min_df': [1, 3],
    'features__root_domain__max_df': [0.5, 0.75],
    'clf__C': [0.1, 1, 10]  # Regularization parameter for LinearSVC
}

In [17]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='f1')

In [ ]:
grid_search.fit(X_train, y_train)

/raid/alebedev/myenvtr/lib/python3.7/site-packages/sklearn/model_selection/_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/raid/alebedev/myenvtr/lib/python3.7/site-packages/sklearn/model_selection/_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If yo

In [ ]:
print("Best parameters:", grid_search.best_params_)

# Optionally, you can access other attributes such as best score and best estimator
print("Best score:", grid_search.best_score_)
print("Best estimator:", grid_search.best_estimator_)

In [ ]:
test_df.fillna("0",inplace=True)

In [77]:
y_predict = pipeline.predict(test_df)

In [ ]:
y_predict = grid_search.predict(test_df)

In [ ]:
X_test = pd.read_csv("test.csv")

In [ ]:
X_test["label"] = y_predict
X_test

In [ ]:
X_test[["ID", "label"]].to_csv("ColumnTransformer_LogisticRegression_grid_search.csv", index=False)

!cat ColumnTransformer_LogisticRegression_grid_search.csv | head

In [4]:
a = pd.read_csv("ColumnTransformerV7.csv", index=False)

TypeError: read_csv() got an unexpected keyword argument 'index'

In [6]:
a["label"] = a["label"].apply(lambda x: int(x))
a

,ID,label
0,135309,0
1,135310,0
2,135311,0
3,135312,1
4,135313,0
...,...,...
165373,300682,0
165374,300683,0
165375,300684,0
165376,300685,0


In [9]:
a.to_csv("ColumnTransformerV7a.csv", index=False)
!cat ColumnTransformerV7a.csv | head

ID,label
135309,0
135310,0
135311,0
135312,1
135313,0
135314,0
135315,0
135316,0
135317,0
cat: write error: Broken pipe
